In [ ]:
!pip install optuna

# Импорт библиотек

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
import optuna
from math import sqrt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Data 1

In [ ]:
df = pd.read_csv('/content/Data_1_stud.csv', sep=';')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Period          64 non-null     object 
 1   Sales_quantity  57 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB


In [ ]:
df['Period'] = pd.to_datetime(df['Period']).dt.normalize()
df.set_index('Period', inplace=True)

In [ ]:
train = df['Sales_quantity'][:'2019-01-09']  # только известные значения
model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
fit = model.fit()
forecast = fit.forecast(steps=7)  # 7 месяцев с 2019-12 по 2020-04
df.loc['2019-01-10':, 'Sales_quantity'] = forecast.values

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
y_test = df['Sales_quantity']['2019-01-03':'2019-01-09']
rmse = root_mean_squared_error(y_pred=forecast, y_true=y_test)
rmse

8554.7076843527

# Data 2

In [ ]:
df = pd.read_csv('/content/Data_2_Stud.csv', sep=';')

In [ ]:
df['Month'] = pd.to_datetime(df['Month']).dt.normalize()
df.set_index('Month', inplace=True)

In [ ]:
df['Value'] = df['Value'].str.replace(',', '.').astype(float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120 entries, 2016-01-01 to 2025-01-12
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   97 non-null     float64
dtypes: float64(1)
memory usage: 1.9 KB


In [ ]:
train = df['Value'][:'2024-01-01']  # только известные значения
model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
fit = model.fit()
forecast = fit.forecast(steps=23)
df.loc['2024-01-02':, 'Value'] = forecast.values

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [ ]:
y_test = df['Value']['2022-01-03':'2024-01-01']

In [ ]:
rmse = root_mean_squared_error(y_pred=forecast, y_true=y_test)
rmse

142.26224533862703

In [ ]:
def objective(trial):
    p = trial.suggest_int('p', 0, 3)
    d = trial.suggest_int('d', 0, 2)
    q = trial.suggest_int('q', 0, 3)
    P = trial.suggest_int('P', 0, 3)
    D = trial.suggest_int('D', 0, 2)
    Q = trial.suggest_int('Q', 0, 3)

    try:
        model = SARIMAX(train,
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, 12),
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        forecast = model_fit.forecast(steps=len(y_test))
        rmse = sqrt(mean_squared_error(y_test, forecast))

    except:
        rmse = float('inf')

    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10, timeout=600)

print("Лучшие параметры:", study.best_trial.params)

[I 2025-06-10 16:25:03,205] A new study created in memory with name: no-name-7a1c3a01-ad32-4bf9-a567-318ba341dd6c
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No

Лучшие параметры: {'p': 3, 'd': 1, 'q': 1, 'P': 0, 'D': 0, 'Q': 3}


In [ ]:
best_params = study.best_trial.params
final_model = SARIMAX(train,
                      order=(best_params['p'], best_params['d'], best_params['q']),
                      seasonal_order=(best_params['P'], best_params['D'], best_params['Q'], 12),
                      enforce_stationarity=False,
                      enforce_invertibility=False)
result = final_model.fit(disp=False)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [ ]:
forecast = result.get_forecast(steps=len(y_test))
forecast_values = forecast.predicted_mean
forecast_values

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


,predicted_mean
97,314.587339
98,350.389766
99,337.379607
100,342.237436
101,381.893708
102,290.270282
103,294.530050
104,383.827886
105,299.795353
106,326.160507


In [ ]:
rmse = root_mean_squared_error(y_pred=forecast_values, y_true=y_test)
rmse

120.0428584403873

# Data 3

In [ ]:
df = pd.read_csv('/content/Data_3_Stud.csv', sep=';')

In [ ]:
df['Date'] = pd.to_datetime(df['YYMMDD'], format='%y%m%d')

In [ ]:
df.drop(columns='YYMMDD', axis=1, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   PRICE   154 non-null    float64       
 1   Date    177 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.9 KB


In [ ]:
df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
df.set_index('Date', inplace=True)

In [ ]:
df

,PRICE
Date,
2022-01-03,346.13
2022-01-10,335.76
2022-01-17,311.60
2022-01-24,329.58
2022-01-31,324.60
...,...
2025-05-12,NaN
2025-05-19,NaN
2025-05-26,NaN


In [ ]:
train = df['PRICE'][:'2024-12-30']  # только известные значения
model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12)
fit = model.fit()
forecast = fit.forecast(steps=23)
df.loc['2024-12-30':, 'PRICE'] = forecast.values

In [ ]:
train = df[-23:]

In [ ]:
y_test = df[-46:-23]

In [ ]:
y_test

,PRICE
131,131.45
132,128.64
133,126.36
134,116.81
135,122.97
136,123.00
137,119.91
138,122.40
139,140.50
140,133.32


In [ ]:
y_pred = forecast
y_pred

,0
97,316.861709
98,357.080482
99,341.482995
100,348.248254
101,393.053945
102,289.663406
103,294.511253
104,344.276568
105,299.918483
106,330.106772


In [ ]:
rmse = root_mean_squared_error(y_pred=y_pred, y_true=y_test)
rmse

142.26224533862703

In [ ]:
def objective(trial):
    p = trial.suggest_int('p', 0, 3)
    d = trial.suggest_int('d', 0, 2)
    q = trial.suggest_int('q', 0, 3)
    P = trial.suggest_int('P', 0, 3)
    D = trial.suggest_int('D', 0, 2)
    Q = trial.suggest_int('Q', 0, 3)

    try:
        model = SARIMAX(train,
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, 12),
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)
        forecast = model_fit.forecast(steps=len(y_test))
        rmse = sqrt(mean_squared_error(y_test, forecast))

    except:
        rmse = float('inf')

    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, timeout=600)

print("Лучшие параметры:", study.best_trial.params)

[I 2025-06-10 16:10:18,980] A new study created in memory with name: no-name-1f2ad1e5-b9bd-4e9d-860d-20713665285c
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:4008: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:175: RuntimeWarning: invalid value encountered

Лучшие параметры: {'p': 0, 'd': 0, 'q': 2, 'P': 1, 'D': 1, 'Q': 0}


In [ ]:
best_params = study.best_trial.params
final_model = SARIMAX(train,
                      order=(best_params['p'], best_params['d'], best_params['q']),
                      seasonal_order=(best_params['P'], best_params['D'], best_params['Q'], 12),
                      enforce_stationarity=False,
                      enforce_invertibility=False)
result = final_model.fit(disp=False)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:4008: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All par

In [ ]:
forecast = result.get_forecast(steps=len(y_test))
forecast_values = forecast.predicted_mean
forecast_values